In [ ]:
!pip3 install opennmt-py

# Function name suggestion

 - get the data
 - explore
 - preprocess
 - train the model
 - predict
 - evaluate the mode

# Data

Data collection & preprocessing pipeline

## 1. Get the data

 1. gitbase+bblfsh `file -> binary UAST [{node: FunctionGroup}, {}, ...]` Q:nested functions?
 2. CodeSearchNet: 3.5m `{func_name: "", code_tokens: ["def", ...]}`

### Create a new dataset (gitbase + bblfsh)

```
.git packfiles -> gitbase -> protobuf [UAST1, UAST2, ...]
                     |
                   bblfsh
```

TODO: visualize the pipeline

#### What is Gitbase?

[github.com/src-d/gitbase](https://github.com/src-d/gitbase)

* Gitbase is **SQL** interface to git repositories
* refrasing: it allows to **query** and **retrieve** required information from **code**

#### In our case it will help with:
* language classification of files
* selecting files from specific programming language
* filtering out vendor & binary files
* parsing files - we don't work with code as raw text - we extract Unified Abstract Syntax Trees (**UAST**)
* extracting function-related parts of **UAST**



#### What is Bblfsh?

http://doc.bblf.sh
    
* it is contanerized parser infrastructure for multiple languages
* it is responsible for extracting Unified Abstract Syntax Trees (**UAST**)
* interactive playground http://play.bblf.sh

TOOD: add logo + an image

In [ ]:
!ls -la ../repos/git-data

for row in query_gitbase("select * from repositories;"):
    print(row)
    
# TODO: save on disk

### Use existing dataset (CodeSearchNet)



A dataset from Github released in 2019, that contains parsed&preprocessed data about functions for multiple languages.

## 2. Explore the data

Fit in RAM on signe machine: `(projet, sha, file, language, func_name, code)`
 * total samples, languages
 * samples/lang
 * seq_len/lang

In [ ]:
# DataFrame

## 3. Preprocess the data
Covert to numbers, as input to the model.

 - save on disk in intermediate format
 - generate the vocabulary (token/sub-token/char)
 - encode using the vocabulary



In [ ]:
# save in plain text
# generate

! onmt_preprocess \
    -train_src data/src-train.txt \
    -train_tgt data/tgt-train.txt \
    -valid_src data/src-val.txt \
    -valid_tgt data/tgt-val.txt
    -save_data data/demo

In [ ]:
# check the results
!head -n 3 data/src-train.txt

# Model

seq2seq model from NMT

## Train

<img src="img/simple-attn-model.png" style="max-width:400px;float:left;"></img>



In [ ]:
# 2-layer LSTM with 500 hidden units
onmt_train -data data/demo -save_model demo-model

# Attention? Transformer?
# to show the advantage of using openNMT

In [ ]:
!ls -la demo-model*

## Predict

In [ ]:
!onmt_translate -model demo-model_XYZ.pt \
    -src data/src-test.txt \
    -output pred.txt -replace_unk -verbose

## Evaluate

In [ ]:
!perl tools/multi-bleu.perl data/src-test.txt < test.pred

In [ ]:
# Learnings:
# - notebook: super-short (preprocess/train/infernece, separate "how to build a dataset") or super-long-and-structured?
# - paths vars should be short (e.g Path.MODEL vs env.path(Files.MODEL_FILE))
# - 